In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import math



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import BatchNormalization

In [37]:
df = pd.read_csv("/home/rahul/Downloads/Tweets.csv")

In [41]:
df['airline_sentiment'] = df['airline_sentiment'].replace('neutral', 1)
df['airline_sentiment'] = df['airline_sentiment'].replace('negative', 0)
df['airline_sentiment'] = df['airline_sentiment'].replace('positive', 2)

In [214]:
X = df['text']
y = df['airline_sentiment']

In [218]:
X[10]

'@VirginAmerica did you know that suicide is the second leading cause of death among teens 10-24'

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
t = Tokenizer()
t.fit_on_texts(X)

In [46]:
# To get total unique words in tweets
vocab_size = len(t.word_index) + 1    


In [49]:
sequences = t.texts_to_sequences(X)

In [51]:
# MAX TWEET SIZE
def max_tweet():
    for i in range(1, len(sequences)):
        max_length = len(sequences[0])
        if len(sequences[i]) > max_length:
            max_length = len(sequences[i])
    return max_length

In [54]:
tweet_num = max_tweet()
tweet_num

30

In [68]:
labels = to_categorical(np.asarray(y))

In [55]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = tweet_num
padded_X = pad_sequences(sequences, padding='post', maxlen=maxlen)

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_X, labels, test_size = 0.2, random_state = 0)

In [70]:
embeddings_index = dict()
f = open('/home/rahul/Downloads/glove.twitter.27B/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [71]:
# Because we're using GloVE, need to create a word embedding/word context matrix
# Create a matrix of weights for words in the training set
# One embedding for each word in the training set

# Get all unique words in our training set: Tokenizer index
# Find the corresponding weight vector in GloVe embedding

# Define size of embedding matrix: number of unique words x embedding dim (100)
embedding_matrix = np.zeros((vocab_size, 100))

# fill in matrix
for word, i in t.word_index.items():  # dictionary
    embedding_vector = embeddings_index.get(word) # gets embedded vector of word from GloVe
    if embedding_vector is not None:
        # add to matrix
        embedding_matrix[i] = embedding_vector # each row of matrix

In [72]:
from tensorflow.keras.layers import Embedding

# input is vocab_size, output is 100
# weights from embedding matrix, set trainable = False
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix],
                           input_length = tweet_num, trainable=False)

In [74]:
lstm_mod1 = Sequential()
lstm_mod1.add(embedding_layer)
lstm_mod1.add(LSTM(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5))
lstm_mod1.add(Dense(3, activation='softmax'))
lstm_mod1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_mod1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1576900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 1,943,239
Trainable params: 366,339
Non-trainable params: 1,576,900
_________________________________________________________________


In [75]:
hist_1 = lstm_mod1.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Epoch 1/100
37/37 [==============================] - 11s 300ms/step - loss: 0.8186 - acc: 0.6375 - val_loss: 0.6913 - val_acc: 0.7277
Epoch 2/100
37/37 [==============================] - 11s 286ms/step - loss: 0.6237 - acc: 0.7519 - val_loss: 0.5813 - val_acc: 0.7746
Epoch 3/100
37/37 [==============================] - 11s 287ms/step - loss: 0.5649 - acc: 0.7716 - val_loss: 0.5634 - val_acc: 0.7759
Epoch 4/100
37/37 [==============================] - 11s 288ms/step - loss: 0.5514 - acc: 0.7812 - val_loss: 0.5467 - val_acc: 0.7806
Epoch 5/100
37/37 [==============================] - 11s 288ms/step - loss: 0.5331 - acc: 0.7860 - val_loss: 0.5419 - val_acc: 0.7840
Epoch 6/100
37/37 [==============================] - 11s 288ms/step - loss: 0.5343 - acc: 0.7844 - val_loss: 0.5246 - val_acc: 0.7917
Epoch 7/100
37/37 [==============================] - 11s 288ms/step - loss: 0.5288 - acc: 0.7904 - val_loss: 0.5189 - val_acc: 0.7930
Epoch 8/100
37/37 [==============================] - 11s 287ms

Epoch 62/100
37/37 [==============================] - 26s 708ms/step - loss: 0.1044 - acc: 0.9625 - val_loss: 0.8483 - val_acc: 0.8007
Epoch 63/100
37/37 [==============================] - 26s 691ms/step - loss: 0.0997 - acc: 0.9646 - val_loss: 0.8866 - val_acc: 0.8109
Epoch 64/100
37/37 [==============================] - 26s 705ms/step - loss: 0.0929 - acc: 0.9682 - val_loss: 0.8696 - val_acc: 0.8118
Epoch 65/100
37/37 [==============================] - 26s 704ms/step - loss: 0.0924 - acc: 0.9680 - val_loss: 0.9076 - val_acc: 0.8109
Epoch 66/100
37/37 [==============================] - 26s 691ms/step - loss: 0.0887 - acc: 0.9690 - val_loss: 0.8650 - val_acc: 0.8050
Epoch 67/100
37/37 [==============================] - 26s 705ms/step - loss: 0.0953 - acc: 0.9657 - val_loss: 0.9552 - val_acc: 0.7968
Epoch 68/100
37/37 [==============================] - 26s 703ms/step - loss: 0.0898 - acc: 0.9666 - val_loss: 0.9062 - val_acc: 0.8054
Epoch 69/100
37/37 [==============================] - 2

In [77]:
lstm_mod1.save("US_sentimental.tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: US_sentimental.tf/assets


In [231]:
rev= '@VirginAmerica I &lt;3 pretty graphics. so much better than minimal iconography. :D'

In [232]:
t.fit_on_texts(rev)


In [233]:
rev_seq=t.texts_to_sequences(rev)

In [234]:
padded_rev = pad_sequences(rev_seq, padding='post', maxlen=maxlen)

In [235]:
class_names = np.array(['Negative', 'Neutral', 'Positive'])

In [236]:
y_pred=lstm_mod1.predict(padded_rev)

In [237]:
np.argmax(y_pred[0])

1

In [238]:
loss, accuracy = lstm_mod1.evaluate(X_test, y_test, verbose=False)

In [208]:
accuracy

0.7930327653884888